# PyTorch: Optimization & Training

https://github.com/pytorch/examples/tree/master/mnist

In [8]:
# torch는 한가지 방법 밖에없음..
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [4]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

#device = 'cuda' if torch.device.is_available() else 'cpu'

# Preprocess

In [43]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081,))
                  ])),
    batch_size=batch_size,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081,))
                  ])),
    batch_size=batch_size,
    shuffle=True)


# Model

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Optimization

- Model과 Optimization 설정

In [22]:
model = Net().to(device)

In [23]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5) # lr : learning ratio

## - parameters들 확인
       - Weight, Bais를 순서대로 보여줌
       

In [24]:
params = list(model.parameters())
for i in range(8):
    print(params[i].size()) 
    
# Convolution의 Weight
# Convolution의 Bais
# Convolution의 Weight
# Convolution의 Bais
# Convolution의 Fully Connected
# Convolution의 Fully Bais
# Convolution의 Fully Connected
# Convolution의 Fully Bais


torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# Before Training

- 학습하기 전에 Model이 Train할 수 있도록 Train Mode로 변환
    - Convolution 또는 Linear 뿐만 아니라, DropOut과 추후에 배우게 될 Batch Normalization과 같이 parameter를 가진 Layer들도 학습하기 위해 준비

In [25]:
#train mode
model.train()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- 모델에 넣기 위한 첫 Batch 데이터 추출

In [28]:
data, target = next(iter(train_loader))

In [29]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- 추출한 Batch 데이터를 cpu 또는 gpu와 같은 device에 compile

In [34]:
data, target = data.to(device), target.to(device)

In [36]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- gradients를 clear해서 새로운 최적화 값을 찾기 위해 준비

In [37]:
optimizer.zero_grad() # 먼저 클리어부터 

- 준비한 데이터를 model에 input으로 넣어 output을 얻음

In [38]:
pred = model(data) # 먼저 예상 tensor 에서도 pred

- Model에서 예측한 결과를 Loss Function에 넣음
    - 여기 예제에서는 Negative Log-Likelihood Loss 라는 Loss Function을 사용

In [46]:
loss = F.nll_loss(pred,target) # target이랑 비교해서 얼마나 틀렸는지

- Back Propagation을 통해 Gradients를 계산

In [40]:
loss.backward() # 기울기 계산

- 계산된 Gradients는 계산된 걸로 끝이 아니라 Parameter에 Update

In [47]:
optimizer.step() # 클리어한 optimizer를 update하기 , 원스텝 지나갔음

# Start Training

위의 최적화 과정을 반복하여 학습 시작

In [48]:
epochs = 1
log_interval = 100 # 몇번 step을 갔는지 보려고 logWLrdjqhfurh

In [45]:
for epoch in range(1, epochs+1):
    # Train Mode
    model.train()
    for batch_idx, (data,target) in enumerate (train_loader):
        data, target = data.to(device), target.to(device) # cpu 나 gpu에 탑재
        optimizer.zero_grad() 
        pred = model(data)
        loss = F.nll_loss(pred, target) # 비교
        loss.backward() # 기울기 계산
        optimizer.step()
        
        # log 찍어보기
        if batch_idx % log_interval == 0 :
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100*batch_idx / len(train_loader), loss.item()))
        

Train Epoch : 1 [0/60000 (0%)]	Loss : 2.296245
Train Epoch : 1 [6400/60000 (11%)]	Loss : 2.213977
Train Epoch : 1 [12800/60000 (21%)]	Loss : 2.128378
Train Epoch : 1 [19200/60000 (32%)]	Loss : 1.950615
Train Epoch : 1 [25600/60000 (43%)]	Loss : 1.743560
Train Epoch : 1 [32000/60000 (53%)]	Loss : 1.277680
Train Epoch : 1 [38400/60000 (64%)]	Loss : 0.965544
Train Epoch : 1 [44800/60000 (75%)]	Loss : 0.642577
Train Epoch : 1 [51200/60000 (85%)]	Loss : 0.491971
Train Epoch : 1 [57600/60000 (96%)]	Loss : 0.582378


# Evaluation

- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

- autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임